In [2]:
from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
from pyexplain.benchmark.check_results import *
import IPython
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'

In [3]:
DISABLE_KILLER_INSTANCES = False

# RQ2 Which domain specific grow improve the overall runtime?

using the cumulative runtime computed directly for the new dataset

In [4]:
mus_output_folders = [
    "MUS/2022030116/"
]

path_mus = [BASE_OUTPUT_PATH + o for o in mus_output_folders]
df_mus = folder_to_pandas_df_pickle(path_mus)

if DISABLE_KILLER_INSTANCES:
    df_mus = df_mus[~df_mus["params_instance"].str.contains("killer")] .reset_index()

df_mus = corr_enhance_df(df_mus)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

df_mus["explanation config"] = df_mus.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_mus["tot_lits_derived"] = df_mus.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_mus["params_grow_config"] = df_mus.apply(lambda row: "MUS", axis=1)
df_mus["time_to_first_expl"] =df_mus.apply(lambda row: row["params_timeout"] if len(row["time_cumul_explain"]) ==0 else row["time_cumul_explain"][0], axis=1)




/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
display(df_mus[["explanation config", 'time_timedout', "time_timedout_initial"]].groupby(by=["explanation config"]).sum())


,time_timedout,time_timedout_initial
explanation config,,
MUS,92,92


In [6]:
all_grows_output_folders = [
    "RQ2_LOGIC_SUDOKU_PUZZLES/2022021411/",
    "RQ2_LOGIC_SUDOKU_PUZZLES_MAXSAT_FULL/2022021708/",
    "RQ2_DEMYSTIFY_PUZZLES/2022021714",
    "RQ2_DEMYSTIFY_PUZZLES_MAXSAT_FULL/2022021708/",
    "RQ2c_DEMYSTIFY_SUDOKU_PUZZLES/2022022508/",
    "RQ2c_DEMYSTIFY_PUZZLES/2022022508/",
    ## re-run up until here!
    "RQ2_MISSING_PUZZLES/2022030215/",
    "RQ2_MISSING_PUZZLES/2022031010/"

]

path_all_grows_output_dirs = [BASE_OUTPUT_PATH + o for o in all_grows_output_folders]

df_all_grows = folder_to_pandas_df_pickle(path_all_grows_output_dirs)

if DISABLE_KILLER_INSTANCES:
    df_all_grows = df_all_grows[~df_all_grows["params_instance"].str.contains("killer")] .reset_index()

## These instances are ignored because their first step derives a lot 100+ literals at once, which doesn't
df_all_grows = corr_enhance_df(df_all_grows)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

##
df_all_grows["tot_lits_derived"] = df_all_grows.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_all_grows["explanation config"] = df_all_grows.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_all_grows["HS"] = df_all_grows.apply(lambda row: row["tot_n_hs"], axis=1)
df_all_grows["time_to_first_expl"] =df_all_grows.apply(lambda row: row["params_timeout"] if len(row["time_cumul_explain"]) ==0 else row["time_cumul_explain"][0], axis=1)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
df_all_grows_filtered = df_all_grows[
    (~df_all_grows["params_grow"].isin(["SAT", "DISABLED"]))&
    (~df_all_grows["explanation config"].isin(["MUS"])) &
    (~(df_all_grows["params_grow"].isin(["CORR_GREEDY", "DISJ_MCS"]) & (df_all_grows["params_interpretation"] == "FULL")))
]

# display(df_all_grows_filtered.groupby(by=["explanation config", "params_grow", "params_interpretation"]).count())

In [8]:
only_sat_grow = [
    "RQ3_DEMYSTIFY_PUZZLES/2022021717/",
    "RQ3_DEMYSTIFY_PUZZLES/2022030117/",
    "RQ3_LOGIC_SUDOKU_PUZZLES/2022021717/",
]

path_only_sat_grow = [BASE_OUTPUT_PATH + o for o in only_sat_grow]

df_only_sat_grow = folder_to_pandas_df_pickle(path_only_sat_grow)

if DISABLE_KILLER_INSTANCES:
    df_only_sat_grow = df_only_sat_grow[~df_only_sat_grow["params_instance"].str.contains("killer")] .reset_index()

## These instances are ignored because their first step derives a lot 100+ literals at once, which doesn't
df_only_sat_grow = corr_enhance_df(df_only_sat_grow)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

##
df_only_sat_grow["tot_lits_derived"] = df_only_sat_grow.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_only_sat_grow["explanation config"] = df_only_sat_grow.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_only_sat_grow["HS"] = df_only_sat_grow.apply(lambda row: row["tot_n_hs"], axis=1)

# display(df_only_sat_grow.groupby(by=["explanation config", "params_grow", "params_interpretation"]).count())

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
sat_corr_folder = [
    #### SAT SUBSETMAX Grow
    "RQ2_SAT_CORR_SUBSETS/2022060214/",
    #### SAT CORR + SUBSETMAX SAT CORR
    "RQ2_SAT_CORR_SUBSETS/2022060210/"    
]

path_sat_corr = [BASE_OUTPUT_PATH + o for o in sat_corr_folder]

df_sat_corr = folder_to_pandas_df_pickle(path_sat_corr)

if DISABLE_KILLER_INSTANCES:
    df_sat_corr = df_sat_corr[~df_sat_corr["params_instance"].str.contains("killer")] .reset_index()

## These instances are ignored because their first step derives a lot 100+ literals at once, which doesn't
df_sat_corr = corr_enhance_df(df_sat_corr)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

##
df_sat_corr["tot_lits_derived"] = df_sat_corr.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_sat_corr["explanation config"] = df_sat_corr.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_sat_corr["HS"] = df_sat_corr.apply(lambda row: row["tot_n_hs"], axis=1)
df_sat_corr["time_to_first_expl"] =df_sat_corr.apply(lambda row: row["params_timeout"] if len(row["time_cumul_explain"]) ==0 else row["time_cumul_explain"][0], axis=1)

# display(df_only_sat_grow.groupby(by=["explanation config", "params_grow", "params_interpretation"]).count())

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
df_all_grows = pd.concat([df_only_sat_grow, df_all_grows_filtered, df_mus, df_sat_corr], axis=0)

# display(df_all_grows.groupby(by=["explanation config", "params_grow", "params_interpretation"]).count())

In [92]:
col_to_display = [
    # 'lits_derived',
    #'max_t_explain',
    # 'min_t_explain',
    #'params_disable_disjoint_mcses',
    #'params_disjoint_mcs_interpretation',
    #'params_disjoint_mcs_weighing',
    #'params_disjoint_mcses',
    #'params_explanation_computer',
    #'params_explanation_config',
    #'params_full_explanation_config',
    #'params_grow',
    #'params_grow_config',
    'params_instance',
    #'params_interpretation',
    #'params_maxsatpolarity',
    #'params_reuse_SSes',
    #'params_sort_literals',
    #'params_timeout',
    #'params_weighing',
    #'time_cumul_explain',
    #'time_explain',
    #'time_postprocessing',
    #'time_preprocess',
    #'time_preseeding',
    #'time_remaining',
    #'time_tavg_greedy_explain',
    #'time_timedout',
    #'time_timedout_initial',
    #'time_timeout',
    'time_to_first_expl',
    #'time_totalTime',
    #'time_totalTime_initial',
    'tot_lits_derived',
    'tot_time_explain',
]

In [93]:
logic_grid_puzzles = [
    'origin-problem',
    'p12', 'p13', 'p16', 'p18', 'p19', 'p20', 'p25', 'p93', 'pastaPuzzle',
]
df_all_grows_lgp = df_all_grows[df_all_grows.params_instance.isin(logic_grid_puzzles)]


In [94]:
best_configuration = (
    (df_all_grows_lgp.params_full_explanation_config == "OUSb+Iterative+Lit. Incr. HS+sort") & 
    (df_all_grows_lgp.params_grow_config == "SAT MCSes + Actual + Unif.")
)

In [98]:
df_lgp_best_config = df_all_grows_lgp[
    best_configuration
][col_to_display].groupby(
    by=[
        #'params_disable_disjoint_mcses',
        #'params_full_explanation_config',
        'params_instance',
        #'params_grow_config',
        #'params_interpretation',
        #'params_maxsatpolarity',
        #'params_reuse_SSes',
        #'params_sort_literals',
        #'params_timeout',
        #'params_weighing'
]).mean().sort_values("tot_time_explain").reset_index()

TypeError: DataFrame.reset_index() got an unexpected keyword argument 'index'

In [102]:
df_lgp_best_config = df_lgp_best_config.set_index("params_instance")
df_lgp_best_config["avg_time_explain"] = df_lgp_best_config["tot_time_explain"]/df_lgp_best_config["tot_lits_derived"]

In [108]:
print(df_lgp_best_config[
    ['time_to_first_expl', 'avg_time_explain', 'tot_time_explain']
].to_latex(float_format="{:.2f}s".format))

\begin{tabular}{lrrr}
\toprule
 & time_to_first_expl & avg_time_explain & tot_time_explain \\
params_instance &  &  &  \\
\midrule
p18 & 1.14s & 0.16s & 23.32s \\
p20 & 1.14s & 0.16s & 23.38s \\
p25 & 1.12s & 0.26s & 38.32s \\
p16 & 1.04s & 0.27s & 40.98s \\
origin-problem & 1.07s & 0.29s & 43.05s \\
p93 & 1.13s & 0.32s & 47.83s \\
p13 & 1.20s & 0.37s & 56.22s \\
p12 & 1.25s & 0.43s & 63.85s \\
p19 & 3.76s & 0.55s & 137.08s \\
pastaPuzzle & 0.60s & 2.98s & 286.03s \\
\bottomrule
\end{tabular}

